In [1]:
from pathlib import Path

In [2]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
  from google.colab import drive
  drive.mount('/content/drive')

In [3]:
if RunningInCOLAB:
  my_file = Path("trainm")
  if my_file.exists()==False:
     LOADED = False
     !unzip drive/"My Drive"/trainm.zip

In [4]:
if False and RunningInCOLAB:
  kaggle_folder = Path(".kaggle")
  if Path("rvdata").exists()==False:
    !pip install -q kaggle
    !mkdir ~/.kaggle
    !cp drive/"My Drive"/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset
    !unzip real-life-violence-situations-dataset.zip
    !rm real-life-violence-situations-dataset.zip
    !rm -R "real life violence situations"
    !mv -T "Real Life Violence Dataset" "rvdata"
    !kaggle datasets download -d vulamnguyen/rwf2000
    !unzip rwf2000.zip
    !rm rwf2000.zip
    !mv RWF-2000/train/* RWF-2000/
    !mv RWF-2000/val/Fight/* RWF-2000/Fight
    !mv RWF-2000/val/NonFight/* RWF-2000/NonFight
    !rm -R RWF-2000/train/
    !rm -R RWF-2000/val/
    #!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git
    #!mv -T A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset vdd
    #!rm -R A-Dataset-for-Automatic-Violence-Detection-in-Videos
    !kaggle datasets download -d toluwaniaremu/smartcity-cctv-violence-detection-dataset-scvd
    !unzip smartcity-cctv-violence-detection-dataset-scvd.zip
    !rm smartcity-cctv-violence-detection-dataset-scvd.zip
    !rm -R SCVD/SCVD_converted_sec_split
    !mkdir SCVD/NonViolence
    !mkdir SCVD/Violence
    !mv SCVD/SCVD_converted/Train/Normal/* SCVD/NonViolence
    !mv SCVD/SCVD_converted/Train/Violence/* SCVD/Violence
    !mv SCVD/SCVD_converted/Train/Weaponized/* SCVD/Violence
    !mv SCVD/SCVD_converted/Test/Normal/* SCVD/NonViolence
    !mv SCVD/SCVD_converted/Test/Violence/* SCVD/Violence
    !mv SCVD/SCVD_converted/Test/Weaponized/* SCVD/Violence
    !rm -R SCVD/SCVD_converted

In [5]:
import numpy as np
mono = False
dtype=np.uint8
max_video_size = 10*1024*1024
channel_first = True
num_classes = 2
frame_readed = 500
frame_pages = 20
nbr_frame = 6
img_width = 320
img_height = 320
img_size = (img_height, img_width)
if channel_first:
  input_shape = (3,) + img_size
else:
  input_shape = img_size + (3,)
full_input_shape = (nbr_frame,) + input_shape
print(full_input_shape)


(6, 3, 320, 320)


In [6]:
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

/home/mac/miniconda3/envs/ai/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mac/miniconda3/envs/ai/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/mac/miniconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
if RunningInCOLAB:
    !pip install torchinfo

In [8]:
import tensorflow as tf
tutils = tf.keras.utils

2024-10-29 05:52:14.071619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730177534.119686   29246 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730177534.134094   29246 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
import os
from torchinfo import summary
import torch.nn as nn
from PIL import Image
import torchvision as tv
from torchvision import *
from sklearn.model_selection import train_test_split
from torchvision.transforms import (
    CenterCrop,
    Normalize,
    RandomCrop,
)
from torchvision.transforms import Compose
import os
import matplotlib.pyplot as plt
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
from torch.utils.data import DataLoader
plt.ion()

import torchvision.models as models
device = "cuda" if torch.cuda.is_available() else "cpu"
import tensorflow.keras.utils as tf_utils

In [10]:
import cv2
def capture(filename):
  frames = np.zeros( (frame_readed,) +input_shape, dtype=dtype)
  i = 0
  vc = cv2.VideoCapture(filename)
  while i<frame_readed:
      if vc.isOpened():
          rval, frame = vc.read()
      else:
          print( 'Failed to read the video ' + filename)
          return frames
      if rval==False:
          break
      frm = cv2.resize(frame, (img_width, img_height))
      #frm = tf.keras.applications.mobilenet.preprocess_input(frm)
      frm = np.expand_dims(frm, axis=0)
      #if np.max(frm)>1:
      #  frm = frm/255.0
      if channel_first:
        frm = frm.transpose(0,3,2,1)
      frames[i][:] = frm
      i +=1
      if i==(nbr_frame*30):
          break
  vc.release()
  step = int(i/nbr_frame)
  if (step>8):
    step=8
  start = int((i - step*nbr_frame)/2)
  resf = np.empty( (nbr_frame,) + input_shape )
  k = 0
  while k<nbr_frame:
      j = int(step*k)
      resf[k][:] = frames[start+j]
      k = k+1
  return resf


In [11]:
def loadData(dataset, ist, ito, fts_train, labs_train):
  print('Loading ' + dataset +' from ' + str(ist) + ' to ' + str(ito) )
  page_size = ito-ist
  i= 0
  for x in os.listdir('./' + dataset):
    lx = x.lower()
    violence = 1 if (lx.startswith('fight') or lx.startswith('viol')) else 0
    td = dataset+'/'+x+'/'
    notGood = 'fight' not in x.lower() and 'viol' not in x.lower()
    if notGood or os.path.isdir(td)==False:
      continue
    n= 0
    #print('-> folder ' +td)
    nbrFiles = 0
    for file in Path(td).rglob("*"):
      fl = os.path.join('', file)
      #print('file ' + fl)
      #print(os.path.getsize(fl))
      if file.is_dir():
        continue
      if os.path.getsize(fl)>max_video_size:
        continue
      nbrFiles += 1
    #print('nbrFiles '+str(nbrFiles))
    if ito > nbrFiles:
      ito = nbrFiles
      ist = nbrFiles-page_size
      print( 'ist - ito ' + str(ist) + ' - ' + str(ito) )
    for file in Path(td).rglob("*"):
      fl = os.path.join('', file)
      if file.is_dir():
        continue
      if os.path.getsize(fl)>max_video_size:
        continue
      #print('visiting ' + fl)
      if n<ist:
        n+=1
        continue
      if n>=ito:
        break
      videos = capture(fl)
      fts_train[i][:][:] = videos
      labs_train[i] = violence
      i+=1
      n+=1
    ###### this part is not usefull anymore
    print("Readed "+str(i))
    half= int(len(fts_train)/2)
    if i<half:
      while i<half:
        print("Random repeat "+str(i)+'/'+str(half))
        p = np.random.randint(0, i-1)
        fts_train[i][:][:] = fts_train[p]
        labs_train[i] = labs_train[p]
        i+=1
    else:
      if i>half:
        while i<len(fts_train):
          print("Random repeat "+str(i)+'/'+str(half))
          p = np.random.randint(half+1, len(fts_train)-1)
          fts_train[i][:][:] = fts_train[p]
          labs_train[i] = labs_train[p]
          i+=1
  #np.save('./data.npy', fts_train)

In [12]:
if RunningInCOLAB:
    !pip install onnx
    !pip install onnxscript

In [13]:
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
import torch.nn.functional as F

debugMode = False
def toCategorical(tensor, classes):
    return F.one_hot(tensor, num_classes=classes)

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X1, X2, X3, X4, X5, X6, y) in enumerate(dataloader):
        # Send data to target device
        X1, X2, X3, X4, X5, X6, y = X1.to(device), X2.to(device),X3.to(device),X4.to(device),X5.to(device),X6.to(device),y.to(device)

        # 1. Forward pass
        y_pred = model(X1, X2, X3, X4, X5, X6)
        if debugMode:
          print("y_pred")
          print(y_pred)
          print("y")
          print(y)
        # 2. Calculate and accumulate loss
        loss = loss_fn( y_pred, y )
        train_loss += loss.item()
        if debugMode:
          print("loss")
          print(loss)

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()
        softmax = torch.softmax(y_pred, dim=0)

        if debugMode:
          print("softmax")
          print(softmax)
        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(softmax, dim=1)
        y_pred_class = toCategorical(y_pred_class, num_classes)
        if debugMode:
          print("y_pred_class")
          print(y_pred_class)
        lgn = (len(y_pred)*num_classes)
        if debugMode :
          print("lgn")
          print(lgn)
        train_acc += (y_pred_class == y).sum().item()/lgn
        if debugMode:
          print("y_pred_class")
          print(y_pred_class)
          print("(y_pred_class == y).sum()")
          print((y_pred_class == y).sum())
          print("(y_pred_class == y).sum().item")
          print((y_pred_class == y).sum().item())
          print("train_acc")
          print(train_acc)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    # Put model in eval mode
    model.eval()
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X1, X2, X3, X4, X5, X6, y) in enumerate(dataloader):
            # Send data to target device
            X1, X2, X3, X4, X5, X6, y = X1.to(device), X2.to(device),X3.to(device),X4.to(device),X5.to(device),X6.to(device),y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X1, X2, X3, X4, X5, X6)
            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            if debugMode:
              print("test_pred_labels")
              print(test_pred_labels)
            test_pred_labels = toCategorical(test_pred_labels, num_classes)
            if debugMode:
              print("test_pred_labels")
              print(test_pred_labels)
            lgn = len(test_pred_logits)*num_classes
            if debugMode:
              print("lgn")
              print(lgn)
            test_acc += ((test_pred_labels == y).sum().item()/lgn)
            if debugMode:
              print("test_acc")
              print(test_acc)
    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def multiInputTrain(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results


In [14]:
all_models = models.list_models()
classification_models = models.list_models(module=models)
# Initialize models
weights = tv.models.EfficientNet_B3_Weights.DEFAULT # .DEFAULT = best available weights
b_size = 20
torch.manual_seed(42)
torch.cuda.manual_seed(42)
class MultiInputs(nn.Module):
  def __init__(self):
    super(MultiInputs, self).__init__()
    cnn = tv.models.efficientnet_b3(weights=weights)
    for param in cnn.features.parameters():
        param.requires_grad = False
    cnn.classifier[1] = nn.Flatten(1)
    self.cnn = cnn
    self.sec = torch.nn.Sequential()
    self.sec.append(torch.nn.Flatten(1)),
    self.sec.append(torch.nn.Linear(9216, 120))
    self.sec.append(torch.nn.ReLU())
    self.sec.append(torch.nn.Dropout(p=0.1))
    self.sec.append(torch.nn.Linear(120, 24))
    self.sec.append(torch.nn.Sigmoid())
    self.sec.append(torch.nn.Dropout(p=0.1))
    self.sec.append(torch.nn.Linear(24, num_classes, bias=True))
  def forward(self, input1, input2, input3, input4, input5, input6):
    c1 = self.cnn(input1).view(-1, 1536)
    c2 = self.cnn(input2).view(-1, 1536)
    c3 = self.cnn(input3).view(-1, 1536)
    c4 = self.cnn(input4).view(-1, 1536)
    c5 = self.cnn(input5).view(-1, 1536)
    c6 = self.cnn(input6).view(-1, 1536)
    all = torch.cat((c1, c2, c3, c4, c5, c6), 1)
    #all = all.view(1, 1, 9216)
    return self.sec(all)
model = MultiInputs().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, labels) -> None:
        super().__init__()

        self.dataset = dataset
        self.labels = labels
        side_size = 310
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        crop_size = 300
        self.transform = Compose([
                    CenterCrop(crop_size),
                    transforms.ToTensor(),
                    Normalize(mean, std),
                ]
            )

    def __len__(self):
        return len(self.dataset)
    def getItem(self, idx):
        return self.__getitem__(idx)
    def __getitem__(self, idx):
        pixels = self.dataset[idx]
        frames=[0, 1, 2, 3, 4, 5];
        for i in range(nbr_frame):
            frm = pixels[i]
            if channel_first:
              frm = frm.transpose(2,1,0)
            pil_img = Image.fromarray(np.uint8(frm)).convert('RGB')
            video_data = self.transform(pil_img)
            frames[i] = video_data
        return frames[0], frames[1], frames[2], frames[3], frames[4], frames[5], self.labels[idx]
vin = (b_size,)+input_shape
#summary(cnn, [vin],
#         col_names=["input_size", "output_size", "num_params"])

#summary(model, [vin, vin, vin, vin, vin, vin],
#        col_names=["input_size", "output_size", "num_params"])
#print(model)

In [23]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Started at =", current_time)
datasets = [ 'trainm', 'SCVD', 'RWF-2000', 'rvdata' ]
dpages = [ 30, 224, 200, 200 ]
dsizes = [ 150, 224, 800, 1000 ]
nbr_it = 3
epochs = 1
LOADED=False
for it in range(nbr_it):
  print('#### IT '+str(it))
  for dt_index in range( len(datasets)):
    dataset = datasets[dt_index]
    page_size = dpages[dt_index]
    size = dsizes[dt_index]
    pages = int(size/page_size)
    for k in range(pages):
      fts_train = np.zeros( (2*page_size,) + full_input_shape, dtype=dtype )
      labs_train = np.empty(2*page_size, dtype=dtype)
      use_previous_load = (it==0 and dt_index==0 and k==0 and LOADED )
      if use_previous_load==False:
        loadData( dataset, k*page_size, (k+1)*page_size, fts_train, labs_train)
      else:
        print("Using previous load")
        LOADED = (it==0 and dt_index==0 and k==0)
      random_state = np.random.randint(1, 100)
      X_train, fights_test, Y_train, labels_test = train_test_split(fts_train,
                          labs_train, test_size=0.2, random_state=random_state)
      y_train = tf_utils.to_categorical(Y_train, num_classes=num_classes)
      y_test = tf_utils.to_categorical(labels_test, num_classes=num_classes)
      from datetime import datetime
      import time
      millis = int(round(time.time() * 1000))
      classification_dataset_train = ClassificationDataset(dataset=X_train, labels=y_train)
      classification_dataset_test = ClassificationDataset(dataset=fights_test, labels=y_test)
      classification_dataloader_train = DataLoader(dataset=classification_dataset_train,
                                                 batch_size=b_size,
                                                 shuffle=True,
                                                 num_workers=0,
                                                 drop_last=False)
      classification_dataloader_test = DataLoader(dataset=classification_dataset_test,
                                             batch_size=b_size,
                                             shuffle=True,
                                             num_workers=0,
                                             drop_last=False)
     # torch.autograd.set_detect_anomaly(True)
      results = multiInputTrain(model=model,
                       train_dataloader=classification_dataloader_train,
                       test_dataloader=classification_dataloader_test,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=epochs,
                       device=device)
      if RunningInCOLAB==False:
        break
    if RunningInCOLAB==False:
        break  
  v1, v2, v3, v4, v5, v6, _ = classification_dataset_test.getItem(0)
  v1 = v1.view(-1, 3, 300, 300).to(device)
  v2 = v2.view(-1, 3, 300, 300).to(device)
  v3 = v3.view(-1, 3, 300, 300).to(device)
  v4 = v4.view(-1, 3, 300, 300).to(device)
  v5 = v5.view(-1, 3, 300, 300).to(device)
  v6 = v6.view(-1, 3, 300, 300).to(device)
  print('Saving ...')
  if RunningInCOLAB:    
      pt = "drive/My Drive/fights"+str(it)+".pt"
      torch.save(model, pt)
      torch.load(pt)
      onnx = "drive/My Drive/fights"+str(it)+".onnx"
      torch_input = torch.randn( (1,) + full_input_shape )
      onnx_program = torch.onnx.export(model, torch_input, onnx)
      #onnx_program.save(onnx)
  else:
      dataFolder = '/home/mac/data/'
      pt = dataFolder + "fights.pt"
      torch.jit.save(model, pt)
      #torch.load(pt)
      torch.jit.load(pt)
      onnx = dataFolder + "fights_pt.onnx"
      torch_input = torch.randn( (1,) + full_input_shape )
      #onnx_program = torch.onnx.export(model, (v1, v2, v3, v4, v5, v6), onnx, opset_version=16)

Started at = 06:09:59
#### IT 0
Loading trainm from 0 to 30
Readed 30
Readed 60


  0%|          | 0/1 [00:45<?, ?it/s]


KeyboardInterrupt: 

In [26]:
cpu_model = model.cpu()
traced_cpu = torch.jit.trace(cpu_model, (v1, v2, v3, v4, v5, v6))
dataFolder = '/home/mac/data/'
pt = dataFolder + "fights.pt"
torch.jit.save(traced_cpu, pt)
#torch.load(pt)
torch.jit.load(pt)
onnx = dataFolder + "fights_pt.onnx"
torch_input = torch.randn( (1,) + full_input_shape )

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/home/mac/miniconda3/envs/ai/lib/python3.10/site-packages/torch/jit/_trace.py:1084: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2 / 2 (100.0%)
Greatest absolute difference: 0.25275176763534546 at index (0, 0) (up to 1e-05 allowed)
Greatest relative difference: 1.7463545294848888 at index (0, 0) (up to 1e-05 allowed)
  _check_trace(
